In [141]:
import pandas as pd #bouger les données (dataframe)
#importer les données nettoyées par Alexandre
censure = pd.read_csv("Base de donnée1", encoding='ISO-8859-1')
complet = pd.read_csv("Base de donnée2", encoding='ISO-8859-1' )

In [142]:
comp=complet.sort_values(by=['num_dossier_mbam_coded', 'date_envoi'])

In [143]:
#on crée de nouvelles variables qui indiquent si renvoie la date des courriels qui ont été lu
i=0
lact=[]#courriel ouvert
lcl=[]#courriel cliqué
llang=[]
while i<len(comp):
    openlink=comp.iloc[i,12]
    clicklink=comp.iloc[i,13]
    if openlink==1:
        lact.append(comp.iloc[i,8])
    else:
        lact.append('2015-01-09T13:03:24') #valeurs arbitraires mises lorsque le courriel n'est pas lu, nous allons ensuite group by selon la date la plus récente,  la valeur est donc très ancienne
    if clicklink==1:
        lcl.append(comp.iloc[i,8])
    else:
        lcl.append('2015-01-09T13:03:24')
    if comp.iloc[i,6]=='FR': #indicateur binaire pour
        llang.append(1)
    else:
        llang.append(0)    
    i+=1
comp['Francais']=llang
comp['Last_opened']=lact
comp['Last_clicked']=lcl
comp['Last_opened'] = comp['Last_opened'].astype('datetime64[ns]')# on convertit les date dans les bons formats
comp['Last_clicked'] = comp['Last_clicked'].astype('datetime64[ns]')
comp['date_envoi'] = comp['date_envoi'].astype('datetime64[ns]')
comp['date_transaction'] = comp['date_transaction'].astype('datetime64[ns]')


testnull=comp['Last_opened'].isnull() 
testnull.sum()
    

0

In [144]:
testnull1=comp['first_transaction'].isnull() #test pour vérifier certaines dates
print(testnull1.sum(), len(testnull1))

print (comp.dtypes)
complet.iloc[586100:586200,19]

2739691 2836381
num_dossier_mbam_coded              object
campaign_id                          int64
campaign_name                       object
campagnes                           object
statut                              object
inscription                        float64
langue                              object
lettre                              object
date_envoi                  datetime64[ns]
hardbounce                           int64
softbounce                           int64
delivered                            int64
open                                 int64
clicked                              int64
converted                            int64
complaint                            int64
unsubscribed                         int64
not_open                             int64
open_but_didnt_click                 int64
first_transaction                   object
date_envoi2                         object
code_promo                          object
point_de_vente                      ob

586100    NaN
586101    NaN
586102    NaN
586103    NaN
586104    NaN
         ... 
586195    NaN
586196    NaN
586197    NaN
586198    NaN
586199    NaN
Name: first_transaction, Length: 100, dtype: object

In [145]:
dic_agg={}#on créé le dicionnaire correspondant aux cellules souhaitées ainsi que leurs cellules d'aggrégation
i=0
somme=['Cadeau','Prolongation','Rabais','hardbounce','softbounce','delivered','open','clicked','complaint','unsubscribed','not_open','open_but_didnt_click']
mini=['date_envoi']
maximum=['converted2','date_transaction','Last_opened', 'Last_clicked'] 
last=['statut', 'Francais']  

#moyenne=['converted']

while i<len(somme):
    dic_agg[somme[i]]='sum' 
    i+=1
i=0
while i<len(mini):
    dic_agg[mini[i]]='min', 'max' 
    i+=1
i=0
while i<len(maximum):
    dic_agg[maximum[i]]='max' 
    i+=1

gbstatut=comp.groupby(['num_dossier_mbam_coded', 'langue', 'statut']).agg(dic_agg)

i=0
while i<len(last):
    dic_agg[last[i]]='last' 
    i+=1
# while i<len(moyenne):
#     dic_agg[moyenne[i]]='mean'
#     i+=1
# i=0

print(dic_agg) # il y a deux group by, l'un ou on ne prend que le id client, et un autre ou on prend aussi le statut pour l'aggréger plus tard
gb=comp.groupby(['num_dossier_mbam_coded',]).agg(dic_agg)

#gb=complet.groupby(['num_dossier_mbam_coded', 'statut']).sum('Cadeau','Prolongation','Rabais','hardbounce','softbounce','delivered','open','clicked','converted','complaint','unsubscribed','not_open','open_but_didnt_click').min('first_transaction').max('date_envoi','Last_opened')
gb.head()

{'Cadeau': 'sum', 'Prolongation': 'sum', 'Rabais': 'sum', 'hardbounce': 'sum', 'softbounce': 'sum', 'delivered': 'sum', 'open': 'sum', 'clicked': 'sum', 'complaint': 'sum', 'unsubscribed': 'sum', 'not_open': 'sum', 'open_but_didnt_click': 'sum', 'date_envoi': ('min', 'max'), 'converted2': 'max', 'date_transaction': 'max', 'Last_opened': 'max', 'Last_clicked': 'max', 'statut': 'last', 'Francais': 'last'}


Cadeau Prolongation Rabais hardbounce  \
                                    sum          sum    sum        sum   
num_dossier_mbam_coded                                                   
0000fce4f46f0e6d5bf322cfe94606b5      1            0      5          0   
00010c8a563eb5555709c8ae3120ba2e      1            6     19          0   
000143d1a4f5b28b0f60e49acdecae5b      1            6     19          0   
00017a72a2b58ac5a63d5ec61085d3fc      1            6     19          0   
0001ea9ccf21b9cf581957136c50fd42      1           12      5          0   

                                 softbounce delivered open clicked complaint  \
                                        sum       sum  sum     sum       sum   
num_dossier_mbam_coded                                                         
0000fce4f46f0e6d5bf322cfe94606b5          0         6    3       1         0   
00010c8a563eb5555709c8ae3120ba2e          0        26    0       0         0   
000143d1a4f5b28b0f60e49acdecae5b          0        26    1       0         0   
00017a72a2b58ac5a63d5ec61085d3fc          0        26    3       0         0   
0001ea9ccf21b9cf581957136c50fd42          0        18   18       0         0   

                                 unsubscribed not_open open_but_didnt_click  \
                                          sum      sum                  sum   
num_dossier_mbam_coded                                                        
0000fce4f46f0e6d5bf322cfe94606b5            0        3                    2   
00010c8a563eb5555709c8ae3120ba2e            0       26                    0   
000143d1a4f5b28b0f60e49acdecae5b            0       25                    1   
00017a72a2b58ac5a63d5ec61085d3fc            0       23                    3   
0001ea9ccf21b9cf581957136c50fd42            0        0                   18   

                                          date_envoi                      \
                                                 min                 max   
num_dossier_mbam_coded                                                     
0000fce4f46f0e6d5bf322cfe94606b5 2020-01-09 11:00:12 2020-02-13 11:30:26   
00010c8a563eb5555709c8ae3120ba2e 2020-01-09 13:03:35 2020-09-24 11:00:38   
000143d1a4f5b28b0f60e49acdecae5b 2020-01-09 13:03:37 2020-09-24 11:00:55   
00017a72a2b58ac5a63d5ec61085d3fc 2020-01-09 13:03:24 2020-09-24 13:03:25   
0001ea9ccf21b9cf581957136c50fd42 2020-01-09 13:03:36 2020-06-25 13:18:41   

                                 converted2 date_transaction  \
                                        max              max   
num_dossier_mbam_coded                                         
0000fce4f46f0e6d5bf322cfe94606b5          0              NaT   
00010c8a563eb5555709c8ae3120ba2e          0              NaT   
000143d1a4f5b28b0f60e49acdecae5b          0              NaT   
00017a72a2b58ac5a63d5ec61085d3fc          0              NaT   
0001ea9ccf21b9cf581957136c50fd42          0              NaT   

                                         Last_opened        Last_clicked  \
                                                 max                 max   
num_dossier_mbam_coded                                                     
0000fce4f46f0e6d5bf322cfe94606b5 2020-02-13 11:30:26 2020-02-13 11:30:26   
00010c8a563eb5555709c8ae3120ba2e 2015-01-09 13:03:24 2015-01-09 13:03:24   
000143d1a4f5b28b0f60e49acdecae5b 2020-01-30 13:03:13 2015-01-09 13:03:24   
00017a72a2b58ac5a63d5ec61085d3fc 2020-09-03 11:00:24 2015-01-09 13:03:24   
0001ea9ccf21b9cf581957136c50fd42 2020-06-25 13:18:41 2015-01-09 13:03:24   

                                     statut Francais  
                                       last     last  
num_dossier_mbam_coded                                
0000fce4f46f0e6d5bf322cfe94606b5  VISITEURS        0  
00010c8a563eb5555709c8ae3120ba2e   INACTIFS        1  
000143d1a4f5b28b0f60e49acdecae5b   INACTIFS        1  
00017a72a2b58ac5a63d5ec61085d3fc   INACTIFS        1  
0001ea9ccf21b9cf581957136c50fd42     ACTIFS       

In [134]:
from datetime import datetime #test de différences de date

now = datetime(2020, 12, 21)
a=(now-gb.iloc[i,-2]).days
print (a)

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'str'

In [135]:
lop=[]#variable du nombre jours depuis la derniere ouverture de courriel
lcld=[]#variable du nombre jours depuis le dernier clic de courriel
lanc=[] #variable d'ancienneté selon le premier message reçu
rod=[] #ratio ouvert/delivered
rco=[] # ratio  clic/ouvert
lcat=[] # 3 catégories correspondant à la variable à la ligne 1
ltest=[]
from datetime import datetime
# print(datetime.now(),gb.iloc[0,-2], (datetime.now()-gb.iloc[0,-2]).dt.days)

now = datetime(2020, 12, 21)#date de la dernière transaction dans le fichier

i=0
while i<len(gb):
    a=(now-gb.iloc[i,-4]).days
    b=(now-gb.iloc[i,-3]).days
    c=(now-gb.iloc[i,-8]).days
    aw=(gb.iloc[i,-9]-gb.iloc[i,-4]).days

    if a>c: # ici, si quelqu'un n'a jamais ouvert de courriel, la date de la dernière activité correspond au premier message reçu
        a=c
    if b>c:
        b=c
    if a<90:
        d='participation_recente'
    elif a<180:
        d='participation_moyenne'
    else:
        d='participation_faible'
    lop.append(a)
    lcld.append(b)
    lanc.append(c)
    lcat.append(d)
    if gb.iloc[i,5]==0:
        rod.append(0)         
    else:
        rod.append(gb.iloc[i,6]/gb.iloc[i,5])
    if gb.iloc[i,6]==0:
        rco.append(0)
    else:
        rco.append(gb.iloc[i,7]/gb.iloc[i,6])
    i+=1

#nouvelles variables
gb['Days_open']=lop
gb['Days_click']=lcld
gb['Anciennete']=lanc
gb['Ratio_ouvert']=rod
gb['Ratio_click']=rco
gb['Groupe_actif']=lcat
#time_d_float = time_d.total_seconds()

gb.dtypes
gb.head()

    


,Cadeau,Prolongation,Rabais,hardbounce,softbounce,delivered,open,clicked,complaint,unsubscribed,...,Last_opened,Last_clicked,statut,Francais,Days_open,Days_click,Anciennete,Ratio_ouvert,Ratio_click,Groupe_actif
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,...,max,max,last,last,,,,,,
num_dossier_mbam_coded,,,,,,,,,,,,,,,,,,,,,
0000fce4f46f0e6d5bf322cfe94606b5,1,0,5,0,0,6,3,1,0,0,...,2020-02-13 11:30:26,2020-02-13 11:30:26,VISITEURS,0,311,311,346,0.500000,0.333333,participation_faible
00010c8a563eb5555709c8ae3120ba2e,1,6,19,0,0,26,0,0,0,0,...,2015-01-09 13:03:24,2015-01-09 13:03:24,INACTIFS,1,346,346,346,0.000000,0.000000,participation_faible
000143d1a4f5b28b0f60e49acdecae5b,1,6,19,0,0,26,1,0,0,0,...,2020-01-30 13:03:13,2015-01-09 13:03:24,INACTIFS,1,325,346,346,0.038462,0.000000,participation_faible
00017a72a2b58ac5a63d5ec61085d3fc,1,6,19,0,0,26,3,0,0,0,...,2020-09-03 11:00:24,2015-01-09 13:03:24,INACTIFS,1,108,346,346,0.115385,0.000000,participation_moyenne
0001ea9ccf21b9cf581957136c50fd42,1,12,5,0,0,18,18,0,0,0,...,2020-06-25 13:18:41,2015-01-09 13:03:24,ACTIFS,1,178,346,346,1.000000,0.000000,participation_moyenne


In [136]:
ar=gbstatut.groupby(['num_dossier_mbam_coded']).size() # test pour déterminer le nombre de gens qui ont eu plus qu'un status
listestatut2=[]
listestatut3=[]
listestatut4=[]
i=0
while i<len(ar):
    if ar.iloc[i]==2:
        listestatut2.append(ar.index[i])
    elif ar.iloc[i]==3:
        listestatut2.append(ar.index[i])
        listestatut3.append(ar.index[i])
    elif ar.iloc[i]==3:
        listestatut2.append(ar.index[i])
        listestatut3.append(ar.index[i])
        listestatut4.append(ar.index[i])

    i+=1
print (len(listestatut2))
print (len(listestatut3))
print (len(listestatut4))
# i=0
# while i<len(ar):
#     if ar.ilo
# print (test)
gb.dtypes



25309
419
0


Cadeau                sum              int64
Prolongation          sum              int64
Rabais                sum              int64
hardbounce            sum              int64
softbounce            sum              int64
delivered             sum              int64
open                  sum              int64
clicked               sum              int64
complaint             sum              int64
unsubscribed          sum              int64
not_open              sum              int64
open_but_didnt_click  sum              int64
date_envoi            min     datetime64[ns]
                      max     datetime64[ns]
converted2            max              int64
date_transaction      max     datetime64[ns]
Last_opened           max     datetime64[ns]
Last_clicked          max     datetime64[ns]
statut                last            object
Francais              last             int64
Days_open                              int64
Days_click                             int64
Anciennete

In [137]:

rrrr=gbstatut.reset_index()
aag=rrrr.sort_values(by=['num_dossier_mbam_coded', ('date_envoi','min')])#on trie les données par date d'envoi pour avoir les statuts dans l'ordre,
output_series = aag.groupby(['num_dossier_mbam_coded'])['statut'].apply(list) # puis on regroupe les dossiers en créant une nouvelle variable qui concat les différents status d'un client

gb.dtypes



Cadeau                sum              int64
Prolongation          sum              int64
Rabais                sum              int64
hardbounce            sum              int64
softbounce            sum              int64
delivered             sum              int64
open                  sum              int64
clicked               sum              int64
complaint             sum              int64
unsubscribed          sum              int64
not_open              sum              int64
open_but_didnt_click  sum              int64
date_envoi            min     datetime64[ns]
                      max     datetime64[ns]
converted2            max              int64
date_transaction      max     datetime64[ns]
Last_opened           max     datetime64[ns]
Last_clicked          max     datetime64[ns]
statut                last            object
Francais              last             int64
Days_open                              int64
Days_click                             int64
Anciennete

In [138]:
print(output_series.head)#exemple

<bound method NDFrame.head of num_dossier_mbam_coded
0000fce4f46f0e6d5bf322cfe94606b5              [VISITEURS]
00010c8a563eb5555709c8ae3120ba2e               [INACTIFS]
000143d1a4f5b28b0f60e49acdecae5b               [INACTIFS]
00017a72a2b58ac5a63d5ec61085d3fc               [INACTIFS]
0001ea9ccf21b9cf581957136c50fd42       [ACTIFS, INACTIFS]
                                            ...          
fffca38fafd4c790f2447b5ee3df38a9    [VISITEURS, INACTIFS]
fffe42ed12574370679a3da5c17be903               [INACTIFS]
fffed5996ecb5832eaf141c9aa589b01                 [ACTIFS]
fffed84f243bac169f1041455d445da4               [INACTIFS]
ffff3a2db7e2e2518c294e1e3ef8ff33                 [ACTIFS]
Name: statut, Length: 140979, dtype: object>


In [139]:
gbfinal = pd.merge(gb, output_series, how="left", on="num_dossier_mbam_coded")#on join la table avec les statuts à l'autre table
print(gbfinal.head())

C:\Users\benoi\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


                                  (Cadeau, sum)  (Prolongation, sum)  \
num_dossier_mbam_coded                                                 
0000fce4f46f0e6d5bf322cfe94606b5              1                    0   
00010c8a563eb5555709c8ae3120ba2e              1                    6   
000143d1a4f5b28b0f60e49acdecae5b              1                    6   
00017a72a2b58ac5a63d5ec61085d3fc              1                    6   
0001ea9ccf21b9cf581957136c50fd42              1                   12   

                                  (Rabais, sum)  (hardbounce, sum)  \
num_dossier_mbam_coded                                               
0000fce4f46f0e6d5bf322cfe94606b5              5                  0   
00010c8a563eb5555709c8ae3120ba2e             19                  0   
000143d1a4f5b28b0f60e49acdecae5b             19                  0   
00017a72a2b58ac5a63d5ec61085d3fc             19                  0   
0001ea9ccf21b9cf581957136c50fd42              5                  0   

    

In [90]:
gb.to_excel('C:/Users/benoi/Documents/HEC/_Hiver 2021/HEC Data Challenge/Db_by_id2.xlsx')

KeyboardInterrupt: 

In [173]:
gbfinal.to_excel('C:/Users/benoi/Documents/HEC/_Hiver 2021/HEC Data Challenge/Db_by_id.xlsx')

In [140]:
gbfinal.to_csv('C:/Users/benoi/Documents/HEC/_Hiver 2021/HEC Data Challenge/Db_by_idfinal8.csv')# on exporte le fichier